Fast Async Code with Cython and AsyncIO
=======================================


_by Stefan Behnel & Anton Caceres_

Skoobe
-------

* eBook subscription service
* 180 000+ books
* Android, iOS apps - 5 stars
* Python based backend

![Alt text](https://staging-cdn.skoobe.net/static/v/12a6f0753eb3c5edbda7861ad83d6593873f4a2a244fc452f5e5d7cf82cb186e.jpg)

Plan of talk
------------

* Introduction to Cython
* Cython and AsyncIO
* Minimal example

# Async Cython Coroutines with Python 3.5

In [1]:
%load_ext cython

In [2]:
import asyncio

In [3]:
def run_async(coro):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(coro)
    return result

## Cython coroutines are fully compatible with asyncio and Python coroutines

In [ ]:
%%cython
# cython: language_level = 3

async def add_one(fut):
    result = await fut
    print("Cy-ADD:", result)
    return result + 1

async def one():
    return 1

In [ ]:
run_async(add_one(add_one(one())))

In [ ]:
async def py_add_one(fut):
    result = await fut
    print("Py-ADD:", result)
    return result + 1

In [ ]:
run_async(add_one(py_add_one(one())))

## Let's play some ping-pong

In [ ]:
%%cython

async def cy_dec1(coro_map, value, show=False):
    if show:
        print('pong')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
async def py_dec1(coro_map, value, show=False):
    if show:
        print('ping')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 6, show=True))

## How fast are Cython coroutines?

In [ ]:
%timeit run_async(py_dec1({0: py_dec1, 1: py_dec1}, 600))  # python-only

In [ ]:
%timeit run_async(cy_dec1({0: cy_dec1, 1: cy_dec1}, 600))  # cython-only

In [ ]:
%timeit run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 600))  # alternate

## -> explain why I/O applications can reduce latency with faster processing intervals

## -> show an example where we receive little data and do a lot of processing on it